<a href="https://colab.research.google.com/github/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes/blob/main/AML_AnomalySegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm -r sample_data/

In [ ]:
# clone the github repo and move all files to the current working directory
!git clone https://github.com/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes.git
!mv Real-Time-Anomaly-Segmentation-for-Road-Scenes/* .
!rm -r Real-Time-Anomaly-Segmentation-for-Road-Scenes/

In [7]:
# download required packages
!pip3 install --quiet cityscapesscripts
!pip3 install --quiet gdown
!pip3 install --quiet numpy
!pip3 install --quiet matplotlib
!pip3 install --quiet Pillow
!pip3 install --quiet torchvision
!pip3 install --quiet visdom
!pip3 install --quiet ood_metrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.6/473.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# download and unzip validation dataset
!gdown 'https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta'
!file 'Validation_Dataset.zip'
!ls -lh Validation_Dataset.zip
!unzip -q 'Validation_Dataset.zip'
!mkdir validation_dataset && cp -pR Validation_Dataset/* validation_dataset/ && rm -R Validation_Dataset/
!unzip -l 'Validation_Dataset.zip'

In [ ]:
# download datasets for training and testing
!csDownload leftImg8bit_trainvaltest.zip
!csDownload gtFine_trainvaltest.zip

# unzip the datasets
!unzip -q 'leftImg8bit_trainvaltest.zip' -d 'cityscapes'
!unzip -o -q 'gtFine_trainvaltest.zip' -d 'cityscapes'

# generate trainIds from labelIds
!CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

## Evaluation

In [5]:
%cd eval

/content/eval


In [9]:
%run evalAnomaly.py --input="../validation_dataset/RoadAnomaly/images/*.png"

Loading model: ../trained_models/erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully


/content/eval/evalAnomaly.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = load_my_state_dict(model, torch.load(weightspath, map_location=lambda storage, loc: st

IndexError: cannot do a non-empty take from an empty axes.